In [55]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

def RMSEvec(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        if ground[i] != 0:
            error += (ground[i] - predict[i])**2
            n += 1
                
    return np.sqrt(error/n)

def meanError(ground_truth,new_res):
    return np.mean(abs((new_res - ground_truth)[ground_truth!=0]))

In [112]:
def dictfromR(R):

    R_dict = {"Users": np.empty([0]), "Movies": np.empty([0]), "Ratings": np.empty([0])}

    for i in range(len(R)):
        for j in range(len(R[0])):
            if R[i,j] != 0:
                R_dict["Users"] = np.append(R_dict["Users"],i)
                R_dict["Movies"] = np.append(R_dict["Movies"],j)
                R_dict["Ratings"] = np.append(R_dict["Ratings"],R[i,j])

    return R_dict

In [182]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/home/marc/Documents/MVA/ProjetGraphes/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5])}


In [183]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [184]:
np.argmax([np.sum(P[i,:] for i in range(len(P)))][0])

321

In [185]:
to_keep = 100
sel = np.where(R[:,321] != 0)[0]
np.random.shuffle(sel)
sel = sel[:len(sel)-to_keep]

In [186]:
ground_truth = copy.deepcopy(R[:,321])
R[sel,321] = 0

In [187]:
np.sum(R[:,321] != 0)

100

In [188]:
R_dictCopy = copy.deepcopy(R_dict)
R_dict = dictfromR(R)

In [189]:
np.where(R_dict['Movies'] == 321)

(array([  102,   513,   589,  6648,  7061,  7135,  7371, 11803, 12347,
        16203, 18478, 19343, 19728, 20029, 20750, 20841, 21151, 21817,
        22605, 22738, 23720, 24493, 24954, 25076, 25123, 26060, 26099,
        27356, 28365, 30368, 30749, 31592, 34790, 34805, 35139, 35905,
        38083, 38275, 40093, 41823, 41936, 42158, 42367, 43025, 44289,
        45135, 46028, 46626, 48227, 48286, 48670, 50112, 50445, 52081,
        52201, 53105, 53604, 56819, 57199, 57801, 58340, 58896, 59106,
        59396, 59944, 60668, 61060, 62869, 63656, 63998, 64617, 64697,
        68760, 70679, 70763, 73047, 73258, 75829, 77362, 80691, 80801,
        81018, 81703, 82249, 84540, 84614, 85054, 86344, 87419, 88682,
        89161, 93357, 95460, 95573, 96811, 98084, 98229, 98298, 99063, 99655]),)

In [190]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [191]:
als.U

array([[ 0.28752542,  1.13707751, -0.60522196,  0.23766157],
       [-0.02806124,  1.55153496, -0.80123162,  0.28348067],
       [-0.37334161,  1.33723987, -0.82683945,  0.49609601],
       ..., 
       [-0.44659387,  1.2304098 , -1.18639409,  0.37842379],
       [ 0.5457621 ,  1.27534422, -1.42277189,  0.07384784],
       [-0.52284374,  1.4390053 , -0.98404174, -0.00499491]])

In [192]:
als.V

array([[-0.47910193,  1.60976707, -1.36481744,  0.22618456],
       [-0.36921288,  1.71992137, -0.73214398, -0.02310704],
       [ 0.20975726,  1.80999905, -0.33176646,  0.24683331],
       ..., 
       [-0.42412092,  1.90592034, -0.84333297, -1.09300086],
       [-0.25447255,  1.14355221, -0.50599978, -0.65580052],
       [ 0.04362086,  1.10270516, -2.36709443, -0.04135227]])

In [193]:
R_rec = np.dot(als.U,np.transpose(als.V))
print((R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec)))
print(R_rec)

[[ 3.16909217  3.07163217  3.10262586 ...,  3.07461405  2.76090659
   3.20897192]
 [ 3.54357354  3.40359887  3.36234727 ...,  3.42953263  2.97385774
   3.51823437]
 [ 3.51060909  3.3259694   3.22593895 ...,  3.26803077  2.87695663
   3.45007107]
 ..., 
 [ 3.62239219  3.36329636  3.18546463 ...,  3.3566069   2.93010231
   3.70113577]
 [ 3.57136287  3.32605981  3.28546026 ...,  3.4239052   2.97048129
   3.92828576]
 [ 3.6256115   3.44788622  3.25371257 ...,  3.58832469  3.06913298
   3.62036849]]
[[ 2.57244881  2.28714377  2.37787489 ...,  2.29587294  1.37752376
   2.68919302]
 [ 3.66870791  3.2589452   3.13818499 ...,  3.33486383  2.0009183
   3.59452987]
 [ 3.57220757  3.03169218  2.7388624  ...,  2.86208228  1.71724937
   3.39498819]
 ..., 
 [ 3.89944213  3.14096335  2.62037763 ...,  3.1213806   1.87282836
   4.12995658]
 [ 3.75005858  3.03195684  2.91310549 ...,  3.31839007  1.99103404
   4.79491689]
 [ 3.90886635  3.38859226  2.82016709 ...,  3.79971272  2.27982763
   3.89351794]]


In [194]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.699193033634
0.888202302247


In [195]:
print(RMSEvec(ground_truth, R_rec[:,321]))

0.78012610852


In [196]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [197]:
np.max(R_rec)

7.932259913429589

In [198]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.         0.5720835  0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 ..., 
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]]


In [199]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [200]:
supp = 100
test_vec = copy.deepcopy(R[:,321])*2
# test_vec[:supp] = [0 for i in range(supp)]
test_vec.shape

(671,)

In [201]:
# test_vec

In [232]:
hfs0, confidence = simple_hfs(als.U, test_vec, L, sim)
# hfs0/2

/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [228]:
confidence.shape

(671,)

In [225]:
print(RMSEvec(ground_truth*(f_u > 0),hfs0/2))
print(RMSEvec(ground_truth, R_rec[:,321]))

ValueError: operands could not be broadcast together with shapes (671,) (571,10) 

In [207]:
print(meanError(ground_truth,hfs0/2))
print(meanError(ground_truth,R_rec[:,321]))

0.480938416422
0.60266724148


In [224]:
f_u > 0

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [ ]:
elmnt = 321
val = []
print(RMSEvec(R[:,elmnt],R_rec[:,elmnt]))
for supp in range(1,671,10):
    test_vec = copy.deepcopy(R[:,elmnt])*2
    test_vec[:supp] = [0 for i in range(supp)]

    hfs0 = simple_hfs(als.U, test_vec, L, sim)
    val.append(RMSEvec(R[:,elmnt],hfs0/2))
    
plt.plot(range(1,671,10),val)

In [50]:
np.sum((hfs0[:100] - P[:100,1]*2)==0)

5

In [149]:
np.sum(hfs0.argmax(axis=0))

14

In [ ]:
hfs0[0][P[0]==0]

In [ ]:
np.min(hfs0[0][P[0]!=0])

In [ ]:
hfs0